# 顧客情報要約システム - DSPy最適化版

このノートブックでは、DSPyのEvaluateとOptimizerを使用して、顧客情報要約プロンプトを最適化します。

## セル1: 環境セットアップ

In [0]:
!pip install dspy-ai openai mlflow pandas numpy

In [0]:
dbutils.library.restartPython()

In [0]:
import dspy
from dspy.evaluate import Evaluate
from dspy.teleprompt import BootstrapFewShot
import mlflow
import pandas as pd
import numpy as np
import os

# MLflow設定
mlflow.openai.autolog()
mlflow.set_experiment(experiment_id=2762244084694129)

print("✓ 環境セットアップ完了")

## セル2: サンプルデータ作成

In [0]:
sample_customers = [
    {
        "id": "C001",
        "name": "山田太郎",
        "age": 35,
        "occupation": "会社員",
        "purchase_history": "過去6ヶ月で3回購入。主に電化製品を購入。平均購入額は5万円。",
        "inquiries": "配送に関する問い合わせが2件。製品の使い方に関する質問が1件。",
        "notes": "リピート顧客。メールマガジン購読中。次回購入時10%割引クーポン保有。"
    },
    {
        "id": "C002",
        "name": "佐藤花子",
        "age": 28,
        "occupation": "自営業",
        "purchase_history": "初回購入。書籍とオフィス用品を購入。購入額は1.2万円。",
        "inquiries": "返品ポリシーに関する問い合わせが1件。",
        "notes": "新規顧客。SNS経由での流入。レビュー投稿の依頼中。"
    },
    {
        "id": "C003",
        "name": "鈴木一郎",
        "age": 42,
        "occupation": "経営者",
        "purchase_history": "VIP顧客。月に2-3回購入。高額商品を好む傾向。累計購入額200万円超。",
        "inquiries": "VIPサポート窓口利用。製品カスタマイズの相談が複数回。",
        "notes": "優良顧客。専任担当者あり。法人契約も検討中。誕生日月に特別オファー送付済み。"
    },
    {
        "id": "C004",
        "name": "田中美咲",
        "age": 31,
        "occupation": "看護師",
        "purchase_history": "過去1年で5回購入。健康関連商品とコスメを主に購入。平均購入額3万円。",
        "inquiries": "製品の成分に関する詳細な問い合わせが3件。アレルギー対応の確認。",
        "notes": "健康志向の高い顧客。定期購入プラン加入中。友人紹介プログラム利用歴あり。"
    },
    {
        "id": "C005",
        "name": "高橋健二",
        "age": 55,
        "occupation": "公務員",
        "purchase_history": "2年前から顧客。年に4-5回購入。趣味関連の商品を購入。平均購入額4万円。",
        "inquiries": "配送遅延に関するクレームが1件。その後のフォローアップで満足度回復。",
        "notes": "安定した購入パターン。ポイントプログラム活用。過去にクレームあったが現在は良好な関係。"
    }
]

print(f"✓ サンプルデータ準備完了: {len(sample_customers)}件")

## セル3: LM設定

In [0]:
lm = dspy.LM(
    model='gpt-4o-mini',
    api_key=os.environ["OPENAI_API_KEY"],
    temperature=0.0,
    max_tokens=500
)

dspy.configure(lm=lm)
print("✓ LM設定完了")

## セル4: シグネチャとモジュール定義

In [0]:
# 入力データクラス(DSPyのExample用)
class CustomerData(dspy.Example):
    def __init__(self, customer_id, name, age, occupation, purchase_history, inquiries, notes):
        super().__init__(
            customer_id=customer_id,
            name=name,
            age=str(age),
            occupation=occupation,
            purchase_history=purchase_history,
            inquiries=inquiries,
            notes=notes
        )

# シグネチャ定義
class CustomerSummary(dspy.Signature):
    """顧客情報を簡潔で分かりやすく一貫性のある要約にします。"""
    
    customer_id: str = dspy.InputField(desc="顧客ID")
    name: str = dspy.InputField(desc="顧客名")
    age: str = dspy.InputField(desc="年齢")
    occupation: str = dspy.InputField(desc="職業")
    purchase_history: str = dspy.InputField(desc="購入履歴")
    inquiries: str = dspy.InputField(desc="問い合わせ履歴")
    notes: str = dspy.InputField(desc="備考")
    
    summary: str = dspy.OutputField(desc="顧客情報の要約(200文字以内)")

# 要約モジュール
class SummaryModule(dspy.Module):
    def __init__(self):
        super().__init__()
        self.generate_summary = dspy.ChainOfThought(CustomerSummary)
    
    def forward(self, customer_id, name, age, occupation, purchase_history, inquiries, notes):
        return self.generate_summary(
            customer_id=customer_id,
            name=name,
            age=age,
            occupation=occupation,
            purchase_history=purchase_history,
            inquiries=inquiries,
            notes=notes
        )

print("✓ モジュール定義完了")

## セル5: データセット準備

In [0]:
# DSPy Exampleオブジェクトに変換
trainset = [
    CustomerData(
        customer_id=c["id"],
        name=c["name"],
        age=c["age"],
        occupation=c["occupation"],
        purchase_history=c["purchase_history"],
        inquiries=c["inquiries"],
        notes=c["notes"]
    )
    for c in sample_customers[:3]  # 最初の3件を訓練用
]

devset = [
    CustomerData(
        customer_id=c["id"],
        name=c["name"],
        age=c["age"],
        occupation=c["occupation"],
        purchase_history=c["purchase_history"],
        inquiries=c["inquiries"],
        notes=c["notes"]
    )
    for c in sample_customers[3:]  # 残りを評価用
]

print(f"✓ データセット準備完了: 訓練={len(trainset)}件, 評価={len(devset)}件")

## セル6: 評価関数定義(LLM-as-a-Judge)

In [0]:
# Judge用のシグネチャ
class QualityAssessment(dspy.Signature):
    """要約の品質を0-10で評価します。"""
    
    summary: str = dspy.InputField(desc="評価対象の要約")
    
    score: int = dspy.OutputField(desc="品質スコア(0-10)")
    reasoning: str = dspy.OutputField(desc="評価理由")

# Judge モジュール
class QualityJudge(dspy.Module):
    def __init__(self):
        super().__init__()
        self.assess = dspy.ChainOfThought(QualityAssessment)
    
    def forward(self, summary):
        return self.assess(summary=summary)

judge = QualityJudge()

# DSPy用評価メトリクス関数
def quality_metric(example, pred, trace=None):
    """
    要約の品質を評価するメトリクス
    戻り値: スコア(0.0-1.0)
    """
    if not pred or not hasattr(pred, 'summary'):
        return 0.0
    
    # LLM-as-a-Judgeで評価
    assessment = judge(summary=pred.summary)
    
    try:
        # スコアを0-1に正規化
        score = int(assessment.score)
        normalized_score = score / 10.0
        return normalized_score
    except:
        return 0.0

print("✓ 評価関数定義完了")

## セル7: ベースラインモデルの評価

In [0]:
# ===== セル7: ベースラインモデルの評価 =====

baseline_model = SummaryModule()

print("=" * 60)
print("ベースラインモデルの評価")
print("=" * 60)

# 手動で評価
baseline_scores = []

for idx, example in enumerate(devset):
    print(f"\n評価中 {idx+1}/{len(devset)}: {example.name}")
    
    # 予測実行
    pred = baseline_model(
        customer_id=example.customer_id,
        name=example.name,
        age=example.age,
        occupation=example.occupation,
        purchase_history=example.purchase_history,
        inquiries=example.inquiries,
        notes=example.notes
    )
    
    # 評価実行
    score = quality_metric(example, pred)
    baseline_scores.append(score)

# 平均スコア計算
baseline_score = sum(baseline_scores) / len(baseline_scores) if baseline_scores else 0.0

print(f"\n{'='*60}")
print(f"ベースライン平均スコア: {baseline_score:.2%}")
print(f"個別スコア: {[f'{s:.2f}' for s in baseline_scores]}")
print(f"{'='*60}")

## セル8: プロンプト最適化(BootstrapFewShot)

In [0]:
print("=" * 60)
print("プロンプト最適化開始")
print("=" * 60)

# BootstrapFewShotオプティマイザー
optimizer = BootstrapFewShot(
    metric=quality_metric,
    max_bootstrapped_demos=2,  # 生成するデモ数
    max_labeled_demos=2,       # 使用するラベル付きデモ数
)

# 最適化実行
optimized_model = optimizer.compile(
    student=SummaryModule(),
    trainset=trainset
)

print("\n✓ 最適化完了")

## セル9: 最適化モデルの評価

In [0]:
# ===== セル9: 最適化モデルの評価 =====

print("=" * 60)
print("最適化モデルの評価")
print("=" * 60)

# 手動で評価
optimized_scores = []

for idx, example in enumerate(devset):
    print(f"\n評価中 {idx+1}/{len(devset)}: {example.name}")
    
    # 予測実行
    pred = optimized_model(
        customer_id=example.customer_id,
        name=example.name,
        age=example.age,
        occupation=example.occupation,
        purchase_history=example.purchase_history,
        inquiries=example.inquiries,
        notes=example.notes
    )
    
    # 評価実行
    score = quality_metric(example, pred)
    optimized_scores.append(score)

# 平均スコア計算
optimized_score = sum(optimized_scores) / len(optimized_scores) if optimized_scores else 0.0

print(f"\n{'='*60}")
print(f"最適化後平均スコア: {optimized_score:.2%}")
print(f"個別スコア: {[f'{s:.2f}' for s in optimized_scores]}")
print(f"{'='*60}")

# 改善率計算
if baseline_score > 0:
    improvement = optimized_score - baseline_score
    improvement_pct = (improvement / baseline_score) * 100
    print(f"\n✓ 改善: {improvement:+.2%} ({improvement_pct:+.1f}%)")
else:
    print(f"\n✓ ベースラインスコアが0のため改善率計算不可")

## セル10: 結果の比較と可視化

In [0]:
# 両モデルで全データの要約を生成
results = []

for customer in sample_customers:
    # ベースライン
    baseline_pred = baseline_model(
        customer_id=customer["id"],
        name=customer["name"],
        age=str(customer["age"]),
        occupation=customer["occupation"],
        purchase_history=customer["purchase_history"],
        inquiries=customer["inquiries"],
        notes=customer["notes"]
    )
    
    # 最適化
    optimized_pred = optimized_model(
        customer_id=customer["id"],
        name=customer["name"],
        age=str(customer["age"]),
        occupation=customer["occupation"],
        purchase_history=customer["purchase_history"],
        inquiries=customer["inquiries"],
        notes=customer["notes"]
    )
    
    # 評価
    baseline_quality = quality_metric(None, baseline_pred)
    optimized_quality = quality_metric(None, optimized_pred)
    
    results.append({
        "customer_id": customer["id"],
        "name": customer["name"],
        "baseline_summary": baseline_pred.summary,
        "baseline_score": baseline_quality,
        "optimized_summary": optimized_pred.summary,
        "optimized_score": optimized_quality,
        "improvement": optimized_quality - baseline_quality
    })

results_df = pd.DataFrame(results)

print("\n" + "=" * 80)
print("詳細比較結果")
print("=" * 80)
print(results_df[["name", "baseline_score", "optimized_score", "improvement"]].to_string(index=False))

## セル11: MLflowに結果を記録

In [0]:
with mlflow.start_run(run_name="customer_summary_optimization"):
    # パラメータ
    mlflow.log_param("model", "gpt-4o-mini")
    mlflow.log_param("temperature", 0.0)
    mlflow.log_param("optimizer", "BootstrapFewShot")
    mlflow.log_param("num_customers", len(sample_customers))
    
    # メトリクス
    mlflow.log_metric("baseline_score", baseline_score)
    mlflow.log_metric("optimized_score", optimized_score)
    mlflow.log_metric("improvement", optimized_score - baseline_score)
    mlflow.log_metric("improvement_pct", (optimized_score - baseline_score) / baseline_score * 100)
    
    # 個別顧客スコア
    for idx, row in results_df.iterrows():
        mlflow.log_metric(f"customer_{idx+1}_baseline", row["baseline_score"])
        mlflow.log_metric(f"customer_{idx+1}_optimized", row["optimized_score"])
    
    # 結果をCSVで保存
    results_df.to_csv("optimization_results.csv", index=False)
    mlflow.log_artifact("optimization_results.csv")
    
    print("\n✓ MLflowに結果を記録完了")

## セル12: サンプル要約の表示

In [0]:
print("\n" + "=" * 80)
print("要約サンプル比較")
print("=" * 80)

for idx in range(min(2, len(results_df))):
    row = results_df.iloc[idx]
    print(f"\n【顧客{idx+1}: {row['name']}】")
    print(f"\n■ベースライン (スコア: {row['baseline_score']:.2f})")
    print(row['baseline_summary'])
    print(f"\n■最適化後 (スコア: {row['optimized_score']:.2f})")
    print(row['optimized_summary'])
    print(f"\n改善: {row['improvement']:+.2f}")
    print("-" * 80)

In [0]:
# ===== 診断セル: 評価関数のテスト =====

print("=" * 60)
print("評価関数の診断")
print("=" * 60)

# テスト用に1件要約を生成
test_customer = sample_customers[0]
test_model = SummaryModule()

test_pred = test_model(
    customer_id=test_customer["id"],
    name=test_customer["name"],
    age=str(test_customer["age"]),
    occupation=test_customer["occupation"],
    purchase_history=test_customer["purchase_history"],
    inquiries=test_customer["inquiries"],
    notes=test_customer["notes"]
)

print(f"\n生成された要約:")
print(test_pred.summary)

print(f"\n評価開始...")

# Judgeで評価
try:
    assessment = judge(summary=test_pred.summary)
    print(f"\nJudge評価結果:")
    print(f"  score: {assessment.score}")
    print(f"  reasoning: {assessment.reasoning}")
    
    # スコア変換テスト
    try:
        score_int = int(assessment.score)
        normalized = score_int / 10.0
        print(f"\n変換後スコア: {normalized}")
    except Exception as e:
        print(f"\nスコア変換エラー: {e}")
        print(f"assessment.scoreの型: {type(assessment.score)}")
        print(f"assessment.scoreの値: {assessment.score}")
        
except Exception as e:
    print(f"\nJudge実行エラー: {e}")
    import traceback
    traceback.print_exc()